In [1]:
!pip install -U iminizinc

Requirement already up-to-date: iminizinc in d:\programs\anaconda\lib\site-packages (0.4)


In [2]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.5.5, build 273041792
Copyright (C) 2014-2021 Monash University, NICTA, Data61


In [3]:
f = open('in.txt', 'r')
lines = f.readlines()
f.close()

In [4]:
width = int(lines[0])
n_plates = int(lines[1])

In [5]:
dims = []
for l in range(n_plates):
    line = lines[l+2].split()
    dims.append([int(line[0]), int(line[1])])
dims

[[3, 3], [2, 4], [2, 8], [2, 5], [4, 7], [4, 5], [2, 4], [1, 9]]

In [6]:
pos = []

In [7]:
%%minizinc -m bind

include "C:/Users/david/Documents/unibo/combinatorial_decision_making_and_optimization/zeynep_kiziltan/project/CDMO/my_cumulative.mzn";

int: n_plates;
set of int: PLATES = 1..n_plates;
array[PLATES,1..2] of int: dims;
array[PLATES] of int: lengths = [dims[i,2] | i in PLATES];

int: width;
set of int: WIDTHS = 1..width;
array[PLATES] of int: widths = [dims[i,1] | i in PLATES];

int: max_length = sum(lengths);
array[PLATES,1..2] of var 0..max_length: pos;
array[int] of var int: x = [pos[p, 1] | p in PLATES];
array[int] of var int: y = [pos[p, 2] | p in PLATES];

var int: tot_length = max(p in PLATES)(pos[p, 1] + lengths[p]);

%constraint cumulative([pos[i, 1] | i in PLATES], lengths, widths, width);
constraint my_cumulative(n_plates, y, x, lengths, widths, width);


solve minimize tot_length;

In [8]:
pos = [[p[1], p[0]] for p in pos]

In [9]:
col_lengths = []
for col in range(width):
    length = 0
    for plate in range(n_plates):
        if pos[plate][0] <= col and pos[plate][0]+dims[plate][0] > col:
            length += dims[plate][1]
            
        col_lengths.append(length)
    
tot_length = max(col_lengths)
tot_length

12

In [10]:
import turtle

In [11]:
import random

def change_color():
    R = random.random()
    B = random.random()
    G = random.random()

    turtle.color((1, 1, 1), (R, G, B))

In [12]:
origin = (width//2, tot_length//2)

for plate in range(n_plates):
    mult = 40
    
    # place to bottom-left corner
    turtle.penup()
    new_pos = (pos[plate][0]-origin[0], pos[plate][1]-origin[1])
    turtle.setpos(new_pos[0]*mult, new_pos[1]*mult)
    turtle.pendown()
    change_color()
    
    # set direction
    turtle.setheading(0)
    
    # start filling
    turtle.begin_fill()
    turtle.forward(dims[plate][0]*mult)
    turtle.setheading(90)
    turtle.forward(dims[plate][1]*mult)
    turtle.setheading(180)
    turtle.forward(dims[plate][0]*mult)
    turtle.setheading(270)
    turtle.forward(dims[plate][1]*mult)
    turtle.end_fill()
    
turtle.done()

In [13]:
output_strings = []
output_strings.append(str(width) + ' ' + str(tot_length))
output_strings.append('\n' + str(n_plates))
for p in range(n_plates):
    output_strings.append('\n' + str(dims[p][0]) + ' ' + str(dims[p][1]) + ' ' + str(pos[p][0]) + ' ' + str(pos[p][1]))

In [14]:
output_strings

['9 12',
 '\n8',
 '\n3 3 0 0',
 '\n2 4 1 3',
 '\n2 8 7 0',
 '\n2 5 5 7',
 '\n4 7 3 0',
 '\n4 5 1 7',
 '\n2 4 7 8',
 '\n1 9 0 3']

In [15]:
# Save results
import os
import json
filename = 'pos.txt'

with open(filename, 'w') as file:
    file.writelines(output_strings)